In [6]:
#Optimizacion Bayesiana de hiperparametros de  rpart

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("rlist")

require("rpart")
require("parallel")

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2360816,126.1,4351382,232.4,4326417,231.1
Vcells,4102417,31.3,95115926,725.7,123760509,944.3


Loading required package: rlist



In [7]:


#Defino la  Optimizacion Bayesiana

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 4000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )             # minbuket NO PUEDE ser mayor que la mitad de minsplit

ksemilla_azar  <- 201107   #cambiar por la primer semilla


In [8]:

#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0( folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}
#------------------------------------------------------------------------------
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 
# particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------
#fold_test  tiene el numero de fold que voy a usar para testear, entreno en el resto de los folds
#param tiene los hiperparametros del arbol

ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],  #entreno en todo MENOS el fold_test que uso para testing
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo, 
                          data[ fold==fold_test, ],  #aplico el modelo sobre los datos de testing
                          type= "prob")   #quiero que me devuelva probabilidades

  prob_baja2  <- prediccion[, "BAJA+2"]  #esta es la probabilidad de baja

  #calculo la ganancia
  ganancia_testing  <- data[ fold==fold_test ][ prob_baja2 > 1/40,  
                                                sum( ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) )] 

  return( ganancia_testing )  #esta es la ganancia sobre el fold de testing, NO esta normalizada
}
#------------------------------------------------------------------------------


In [9]:

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos

  particionar( data, divi, seed=semilla, agrupa=pagrupa )  #particiono en dataset en folds

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a qfolds si posee Linux o Mac OS

  data[ , fold := NULL ]

  #devuelvo la primer ganancia y el promedio
  ganancia_promedio  <- mean( unlist( ganancias ) )   #promedio las ganancias
  ganancia_promedio_normalizada  <- ganancia_promedio * qfolds  #aqui normalizo la ganancia
  
  return( ganancia_promedio_normalizada )
}
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <- ArbolesCrossValidation( dataset,
                                        param= x, #los hiperparametros del arbol
                                        qfolds= xval_folds,  #la cantidad de folds
                                        pagrupa= "clase_ternaria",
                                        semilla= ksemilla_azar )

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   xx$iteracion <- GLOBAL_iteracion
   loguear( xx,  arch= archivo_log )

   return( ganancia )
}
#------------------------------------------------------------------------------
#Aqui empieza el programa


In [10]:

setwd(".\\")   #Establezco el Working Directory

#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")
#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]


#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/HT3210/", showWarnings = FALSE )
setwd("./exp/HT3210/")   #Establezco el Working Directory DEL EXPERIMENTO


archivo_log  <- "HT321.txt"
archivo_BO   <- "HT321.RDATA"

#leo si ya existe el log, para retomar en caso que se se corte el programa
GLOBAL_iteracion  <- 0

if( file.exists(archivo_log) )
{
 tabla_log  <- fread( archivo_log )
 GLOBAL_iteracion  <- nrow( tabla_log )
}



#Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar  <- EstimarGanancia

configureMlr( show.learner.output= FALSE)

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= archivo_BO)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

#inicio la optimizacion bayesiana
if( !file.exists( archivo_BO ) ) {

  run  <- mbo( fun=     obj.fun, 
               learner= surr.km,
               control= ctrl)

} else  run  <- mboContinue( archivo_BO )   #retomo en caso que ya exista



Computing y column(s) for design. Not provided.



20230415 083114	-0.66637015613378	4513	1678	7	5	50901000	1
20230415 083243	-0.0708840988969314	5365	97	16	5	50673000	2
20230415 083341	-0.727520458123763	6361	2338	4	5	50055000	3
20230415 083540	-0.0157652133115334	3783	854	15	5	51303000	4
20230415 083718	-0.266948124789633	7520	2244	18	5	50055000	5
20230415 083908	-0.918727741073235	6701	1048	17	5	50055000	6
20230415 084129	-0.859220127915614	2153	312	19	5	52977000	7
20230415 084255	-0.366234884755977	7166	1333	8	5	50055000	8
20230415 084438	-0.764191181046772	5216	1970	12	5	50862000	9
20230415 084536	-0.130531295898254	4525	648	4	5	50454000	10
20230415 084724	-0.685466156963958	7044	1407	14	5	50055000	11
20230415 084824	0.0420622585457751	6593	2099	9	5	0	12
20230415 084955	-0.556921265029814	6654	687	17	5	50055000	13
20230415 085036	-0.82350544721121	3351	1666	11	5	52485000	14
20230415 085112	-0.528771402977873	7448	3503	19	5	49716000	15
20230415 085154	-0.255266323965043	3304	392	9	5	52797000	16


[mbo] 0: cp=-0.666; minsplit=4513; minbucket=1678; maxdepth=7 : y = 5.09e+07 : 31.4 secs : initdesign

[mbo] 0: cp=-0.0709; minsplit=5365; minbucket=97; maxdepth=16 : y = 5.07e+07 : 89.3 secs : initdesign

[mbo] 0: cp=-0.728; minsplit=6361; minbucket=2338; maxdepth=4 : y = 5.01e+07 : 57.9 secs : initdesign

[mbo] 0: cp=-0.0158; minsplit=3783; minbucket=854; maxdepth=15 : y = 5.13e+07 : 118.3 secs : initdesign

[mbo] 0: cp=-0.267; minsplit=7520; minbucket=2244; maxdepth=18 : y = 5.01e+07 : 98.6 secs : initdesign

[mbo] 0: cp=-0.919; minsplit=6701; minbucket=1048; maxdepth=17 : y = 5.01e+07 : 109.8 secs : initdesign

[mbo] 0: cp=-0.859; minsplit=2153; minbucket=312; maxdepth=19 : y = 5.3e+07 : 141.5 secs : initdesign

[mbo] 0: cp=-0.366; minsplit=7166; minbucket=1333; maxdepth=8 : y = 5.01e+07 : 85.7 secs : initdesign

[mbo] 0: cp=-0.764; minsplit=5216; minbucket=1970; maxdepth=12 : y = 5.09e+07 : 103.3 secs : initdesign

[mbo] 0: cp=-0.131; minsplit=4525; minbucket=648; maxdepth=4 : y =

20230415 085309	-0.105689263504282	3591	1	20	5	51924000	17


[mbo] 1: cp=-0.106; minsplit=3591; minbucket=1; maxdepth=20 : y = 5.19e+07 : 72.3 secs : infill_ei



...

[mbo] 98: cp=-0.0515; minsplit=1730; minbucket=862; maxdepth=6 : y = 5.39e+07 : 25.2 secs : infill_ei



20230415 104014	-0.742121191256517	3530	4	20	5	52104000	115


[mbo] 99: cp=-0.742; minsplit=3530; minbucket=4; maxdepth=20 : y = 5.21e+07 : 57.1 secs : infill_ei



20230415 104047	-0.381886563712102	5292	2639	20	5	50256000	116


[mbo] 100: cp=-0.382; minsplit=5292; minbucket=2639; maxdepth=20 : y = 5.03e+07 : 32.0 secs : infill_ei

Saved the final state in the file HT321.RDATA

